In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [2]:
# For OpenAI after 1.0.0 (1.13.3 at the time of writing)
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

![](../images/06-iterative.png)

![](../images/07-process.png)

## Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our Mid-Century Modern Office Chair, a perfect blend of style and functionality designed to complement any office setting. This chair is part of an exquisite collection that includes a range of office furniture, from sleek filing cabinets to elegant meeting tables, all inspired by the timeless mid-century aesthetic.

Choose from a variety of shell colors and base finishes to match your office decor. Our chair comes in two models: the SWC-100, featuring a plastic back with front upholstery, and the SWC-110, offering full upholstery with a selection of 10 fabric and 6 luxurious leather options. The base finishes include sophisticated stainless steel, contemporary matte black, sleek gloss white, or a classic chrome look.

Customize your comfort with options for armrests, ensuring that your chair fits your personal preference and workspace requirements. Whether you're outfitting a home office or a business environment, this chair is built to meet the demands of 

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [5]:
len(response)

2457

In [6]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Experience Italian craftsmanship with our mid-century inspired office chair, featuring a durable 10mm thick shell and a 5-wheel coated aluminum base for smooth mobility. Choose from plastic or fully upholstered options in a variety of fabrics and leathers, complemented by stainless steel, matte black, gloss white, or chrome finishes. Tailor comfort with adjustable seat height, optional armrests, and select foam densities. Suitable for both home and contract use.


Try to say "Use at most 3 sentences" or "Use at most 280 characters"

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [7]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Discover our mid-century inspired office chair, crafted with a robust 5-wheel coated aluminum base and a 10 mm thick shell. Choose from plastic or fully upholstered in premium fabrics or leathers, complemented by stainless steel, matte black, gloss white, or chrome finishes. Features pneumatic height adjustment and optional armrests for tailored comfort. Made in Italy.


In [8]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Discover our mid-century inspired office chair, crafted with a 10 mm thick shell and a durable cast aluminum base coated in modified nylon PA6/PA66. Choose from a stainless steel, matte black, gloss white, or chrome finish, and customize with or without armrests. The seat is padded with HD36 foam, ensuring comfort and support. This versatile chair is available in both plastic back with front upholstery (SWC-100) or fully upholstered (SWC-110) versions, with a range of 10 fabric and 6 leather options. Ideal for both home and office environments, this Italian-made chair meets contract use standards and offers pneumatic height adjustment for optimal ergonomics. Product IDs: SWC-100, SWC-110.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

```html
<div>
  <p>Introducing our exquisite line of mid-century inspired office furniture, designed to elevate any workspace with its timeless elegance and modern functionality. This versatile chair, available in both the SWC-100 and SWC-110 models, caters to a variety of aesthetic preferences and practical needs. Choose from a palette of 10 fabric and 6 leather upholstery options to seamlessly blend with your office decor. The chair's base is offered in four sophisticated finishes: stainless steel, matte black, gloss white, or chrome, ensuring a perfect match for your style.</p>
  
  <p>Constructed with a durable 5-wheel plastic coated aluminum base, the chair provides stability and ease of movement. The pneumatic adjustment feature allows for effortless height customization, ensuring optimal comfort during long work sessions. Whether you opt for the armless design or the 8 position PU armrests, this chair is built to accommodate your personal comfort needs.</p>
  
  <p>Designed for 

## Load Python libraries to view HTML

In [10]:
from IPython.display import display, HTML

In [11]:
display(HTML(response))

Dimension,Measurements (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


## Try experimenting on your own!